In [1]:
import cv2
import numpy as np
import random

In [2]:
def make_noise(image, prob):
    r, g, b = cv2.split(image)
    def make_noise_channel(channel):
        channel_prob = prob / image.shape[2]
        result_channel = channel.copy()
        for i in range(channel.shape[0]):
            for j in range(channel.shape[1]):
                rand = np.random.randint(0, 101)
                if rand <= channel_prob:
                    result_channel[i, j] = np.random.randint(0, 256)
        return result_channel
    
    result_r = make_noise_channel(r)
    result_g = make_noise_channel(g)
    result_b = make_noise_channel(b)
    
    result = cv2.merge((result_r, result_g, result_b))
    
    return result

def get_error(image, filtered_image):
    difference = (image - filtered_image) ** 2
    
    return (np.sum(difference) / (image.shape[0] * image.shape[1])) ** 0.5

def filter_image(image, mask, rang, suffix):
    def filter_channel(channel):
        def filter_elem(i, j):
            image_mask_value_mult = []
            for k in range(mask.shape[0]):
                for l in range(mask.shape[1]):
                    for m in range(mask[k, l]):
                        image_mask_value_mult.append(channel[i - offset + k, j - offset + l])
            
            return np.sort(np.array(image_mask_value_mult))[rang]
    
        copy_channel = channel.copy()
        offset = int((mask.shape[0] - 1) / 2)
        for i in range(offset, channel.shape[0] - offset):
            for j in range(offset, channel.shape[1] - offset):
                copy_channel[i, j] = filter_elem(i, j)
        return copy_channel
    
    r, g, b = cv2.split(image)
    
    cv2.imwrite(path + suffix + 'r.jpg', r)
    cv2.imwrite(path + suffix + 'g.jpg', g)
    cv2.imwrite(path + suffix + 'b.jpg', b)
    
    r_filtered = filter_channel(r)
    g_filtered = filter_channel(g)
    b_filtered = filter_channel(b)
    
    cv2.imwrite(path + suffix + 'r_filtered.jpg', r_filtered)
    cv2.imwrite(path + suffix + 'g_filtered.jpg', g_filtered)
    cv2.imwrite(path + suffix + 'b_filtered.jpg', b_filtered)
    
    result = cv2.merge((r_filtered, g_filtered, b_filtered))
    
    return result


In [3]:
mask = np.array([[2, 1, 2],
                 [1, 1, 1],
                 [2, 1, 2]])
path = '/home/sergei/'
image = cv2.imread(path + 'bicycle.jpg')
noise_image = make_noise(image, 9)
cv2.imwrite(path + 'bicycle_noise.jpg', noise_image)

True

# Осуществить взвешенную медианную фильтрацию входного изображения, совпадающего с одним из тестовых изображений лабораторной работы 1, соответствующим окном. Виды окон по вариантам приведены ниже

In [4]:
image_filtered_median = filter_image(noise_image, mask, 4, 'median')
cv2.imwrite(path + 'bicycle_filter_median.jpg', image_filtered_median)

True

# Осуществить взвешенную ранговую фильтрацию входного изображения соответствующим окном со значением ранга 1, N, r, где N = S∙S, где S – размер маски, r – экспериментально выбранный ранг. Виды окон по вариантам приведены ниже.

In [5]:
image_filtered_one = filter_image(noise_image, mask, 1, 'one')
cv2.imwrite(path + 'bicycle_filter_one.jpg', image_filtered_one)

True

In [6]:
image_filtered_N = filter_image(noise_image, mask, 8, 'N')
cv2.imwrite(path + 'bicycle_filter_N.jpg', image_filtered_N)

True

In [7]:
image_filtered_s = filter_image(noise_image, mask, 6, 's')
cv2.imwrite(path + 'bicycle_filter_s.jpg', image_filtered_s)

True

# Вычислить ошибку и сравнить результат с результатом лабораторной работы 1 соответствующего входного изображения.

In [12]:
print('Error for median {}'.format(get_error(image, image_filtered_median)))

Error for median 14.470466290574974


In [13]:
print('Error for rang = 1 {}'.format(get_error(image, image_filtered_one)))

Error for rang = 1 15.882185911685246


In [14]:
print('Error for rang = N {}'.format(get_error(image, image_filtered_N)))

Error for rang = N 14.46403189524507


In [15]:
print('Error for rang = r {}'.format(get_error(image, image_filtered_s)))

Error for rang = r 13.453406012357366
